In [115]:
import numpy as np
import random
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display
import ipywidgets as widgets
symbols = ['B','.']
data_length = 10 # set it to >= 4

# generate random data
data = np.array([random.choice(symbols) for _ in range(data_length)])
B_count = np.sum(data == 'B')
print(data)

['.' 'B' '.' 'B' 'B' '.' 'B' '.' '.' '.']


In [116]:
def get_longest_streak(data):
    streaks = []
    i = 0
    max_score = 0
    while i < len(data):
        symbol = data[i]
        if symbol == 'B':
            score = 1
            start_pos = i
            while i < len(data):
                try:
                    if data[i+2] == 'B' and data[i+1] == '.':
                        score += 1
                        i += 2
                    else:
                        if score > max_score:
                            max_score = score
                        streaks.append({'score': score, 'start_pos': start_pos, 'end_pos': i})
                        break
                except IndexError:
                    if score > max_score:
                        max_score = score
                    streaks.append({'score': score, 'start_pos': start_pos, 'end_pos': i})
                    break
        i += 1

    # get all streaks with max_score
    max_score_streaks = [streak for streak in streaks if streak['score'] == max_score]
    
    for streak in max_score_streaks:
        # prevent zero length streaks
        if streak['start_pos'] == streak['end_pos']:
            streak['start_pos'] = 0
            streak['end_pos'] = -1

    if len(max_score_streaks) <= 2:
        return max_score_streaks[0]
    
    distances = []
    for i, streak in enumerate(max_score_streaks):
        if i + 1 < len(max_score_streaks):
            distances.append({
                'distance': max_score_streaks[i+1]['start_pos'] - streak['end_pos'],
                'streak' : streak
            })
    
    # select streak with less distance between neighbour streak
    closest_streak = min(distances, key=lambda x: x['distance'])['streak']
    return closest_streak
        
longest_streak = get_longest_streak(data)
print(longest_streak) 

{'score': 2, 'start_pos': 1, 'end_pos': 3}


In [117]:
def swaps_to_streak(side_data, B_even):
    '''
    Input: np.array of side data, example: ['B', '.', '.', '.', '.', '.', 'B', 'B', '.']
           B_even: Bool, should B be placed in even positions like [0,2,4,6...]
    Output: number of swaps needed to continue the streak, for the example above it will be a 3, to make a streak like ['B', '.', 'B', '.', 'B', '.'...]
    '''
    B_even = not B_even # swap values, to match the pattern

    B_indices = np.where(side_data == 'B')[0] # array of B's that were not used in swap
    correct_Bs = B_indices[B_indices % 2 == B_even] # use correct B's if incorrent ran out
    incorrect_Bs = B_indices[B_indices % 2 != B_even] # use incorrect B's first
    B_indices = np.concatenate((correct_Bs, incorrect_Bs))
    
    dot_count = len(side_data) - len(B_indices)
    swap_count = 0 # swaps done
    swaps = [] # indices of swaps
    if dot_count < len(B_indices):
        # add N imaginary dots, they are already on other side
        imaginary_dots = np.array(['.'] * (len(B_indices) - dot_count))
        side_data = np.concatenate((side_data, imaginary_dots))

    print("B's position must be even: ",not B_even)
    print('Processing side:', side_data)
    
    for i in range(B_even, len(side_data), 2):
        if not len(B_indices):
            # no more B's to swap
            break
        

        if side_data[i] == 'B':
            if i not in B_indices:
                # skip because this B was used in swap
                continue
            # B in correct position, prevent it from swapping in future
            B_indices = B_indices[B_indices != i]

        elif side_data[i] == '.':
            # dot is in incorrect position, so swap it
            swaps.append([i, B_indices[-1]])

            B_indices = np.delete(B_indices, -1)
            swap_count += 1
    print('Swaps to make a streak on this side:', swap_count)
    return (swap_count, np.array(swaps))



# side = np.array(['B', '.', 'B', 'B', '.', '.', 'B', '.', '.', 'B'])
left_side = np.flip((data[:longest_streak['start_pos']]))
right_side = data[longest_streak['end_pos']+1:]
print(swaps_to_streak(left_side, longest_streak['start_pos'] % 2 == 0)) # process left side
print(swaps_to_streak(right_side, longest_streak['start_pos'] % 2 == 0)) # process right side



B's position must be even:  False
Processing side: ['.']
Swaps to make a streak on this side: 0
(0, array([], dtype=float64))
B's position must be even:  False
Processing side: ['B' '.' 'B' '.' '.' '.']
Swaps to make a streak on this side: 2
(2, array([[1, 2],
       [3, 0]], dtype=int64))


In [118]:
def place_buckets(data, longest_streak):
    '''
    Input: np.array data, which has only two unique values B and . Example [. . B . B . . B]
    Output: number of iterations to place B in the order like [B . B . B . . .], it is 3 in this case.
    Function should return -1 if it is impossible.
    So it has to be a . between two B.
    '''
    B_count = np.sum(data == 'B')
    D_count = np.sum(data == '.')
    if D_count - B_count < 0 or B_count < 2:
        # it is impossible to sort, not enough space
        return -1, []
    
    swap_count = 0 # init swap count 
    left_side = data[:longest_streak['start_pos']]
    right_side = data[longest_streak['end_pos']+1:]
    B_even = longest_streak['start_pos'] % 2 == 0

    left_side = np.flip(left_side, axis=0) # flip the side to be equal positioned to right_side

    left_side_swap_count, left_side_swaps = swaps_to_streak(left_side, B_even)
    right_side_swap_count, right_side_swaps = swaps_to_streak(right_side, B_even)

    left_side_swaps = np.abs(left_side_swaps - longest_streak['start_pos'] + 1) # shift indices to match the original data
    right_side_swaps = right_side_swaps + longest_streak['end_pos'] + 1 # shift indices to match the original data
    if(left_side_swaps.ndim != right_side_swaps.ndim):
        # make them same dimensions
        if left_side_swaps.ndim > right_side_swaps.ndim:
            swaps = left_side_swaps
        else:
            swaps = right_side_swaps
    else:
        swaps = np.concatenate([left_side_swaps,right_side_swaps])

    swap_count += left_side_swap_count + right_side_swap_count

    return (swap_count, swaps)


In [119]:
def visualize(data, swaps, iter):
    '''
    Input: swaps - list of tuples with indices of elements to swap like (1,2)
    Output: None
    '''

    plt.figure()
    # plt.axis('off')
    # hide y axis
    plt.yticks([])
    plt.xticks([x for x in range(len(data))])
    # make size normal
    plt.xlim(-1, len(data))
    plt.ylim(-1, 1)
    visual_data = np.array(data, dtype=object)

    for i, symbol in enumerate(visual_data):
        # replace B with green, .(D) with blue
        if symbol == 'B':
            visual_data[i] = {'color': 'green', 'symbol': 'B'}
        else:
            visual_data[i] = {'color': 'blue', 'symbol': 'D'}

    if iter != 0:
        print("Iter:"   , iter)
        for i in range(iter):
            # swap and color last swap red
            if i == iter - 1:
                print("Swapping:", swaps[i])
                visual_data[swaps[i][0]]['color'] = 'red'
                visual_data[swaps[i][1]]['color'] = 'red'
            visual_data[swaps[i][0]], visual_data[swaps[i][1]] = visual_data[swaps[i][1]], visual_data[swaps[i][0]]
    else:
        print("Initial data")

    for i, symbol in enumerate(visual_data):
        # plot symbols
        plt.text(i, 0, symbol['symbol'], horizontalalignment='center', verticalalignment='center', color=symbol['color'])

    plt.show()


In [120]:
longest_streak = get_longest_streak(data)
print("The longest streak of B's:", longest_streak)
print('Current data:\n', data)
print("="*10, " RESULT ", "="*10)
swap_count, swaps = place_buckets(data, longest_streak)
print('Required swaps:', swap_count)
if(swap_count == -1):
    print('It is impossible to sort this data')
else:
    # int slider interactive 
    iter_visual =  interactive(visualize, data=widgets.fixed(data), swaps=widgets.fixed(swaps), iter=widgets.IntSlider(min=0, max=swap_count, step=1, value=0))
    display(iter_visual)



The longest streak of B's: {'score': 2, 'start_pos': 1, 'end_pos': 3}
Current data:
 ['.' 'B' '.' 'B' 'B' '.' 'B' '.' '.' '.']
==========  RESULT  ==========
B's position must be even:  False
Processing side: ['.']
Swaps to make a streak on this side: 0
B's position must be even:  False
Processing side: ['B' '.' 'B' '.' '.' '.']
Swaps to make a streak on this side: 2
Required swaps: 2


interactive(children=(IntSlider(value=0, description='iter', max=2), Output()), _dom_classes=('widget-interact…